In [1]:
# !gdown 1yVbhJWh4L1unDbDT4APOusTXlwic7aE9
# !gdown 1-F1DO6UNkz3ndjSV4kSzu7zcaQJtf5De
# !gdown 1LouWfzlgol2J2EdqFy0YVIPh93uSq2ON

In [1]:
import os 
from dotenv import load_dotenv # python -m pip install python-dotenv
load_dotenv()

True

In [2]:
from langchain.document_loaders import PyPDFLoader 
# pip install --upgrade langchain
# pip install -U langchain-community
# pip install pypdf

In [9]:
loaders = [
    PyPDFLoader("PA - Consolidated lecture notes.pdf"),
    # PyPDFLoader("SQL Revision Notes.pdf"),
    PyPDFLoader("PA - Consolidated lecture notes.pdf"),
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1500,
    chunk_overlap=150,
)

In [11]:
splits = text_splitter.split_documents(docs)

In [4]:
# Embeddings

# from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# model_name = "BAAI/bge-small-en"
# model_kwargs = {"device": "cpu"}
# encode_kwargs = {"normalize_embeddings": True}
# embedding = HuggingFaceBgeEmbeddings(
#     model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
# )

from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_BASE_URL"),
    api_version = "2023-03-15-preview",
    model="text-embedding-3-small",
    )

In [19]:
from langchain_community.vectorstores import FAISS # pip install faiss-cpu

db = FAISS.from_documents(
    splits,
    embedding
)

In [20]:
query = "what is discussed about metrics "
docs = db.similarity_search_with_score(query)

In [21]:
docs

[(Document(metadata={'source': 'PA - Consolidated lecture notes.pdf', 'page': 20}, page_content='●\nThere\nare\nmany\ndifferent\nmetrics\nyou\ncould\nbe\ntracking\n(e.g.,\nimpressions,\nclicks,\nreturn\non\nad\nspend,\netc.)\nand\nyour\ninterviewer\nwill\nwant\nto\nhear\nyou\nselect\nthe\nmost\nimportant\nones\nusing\na\nrigorous\nprocess.\n2.\nMetric\nchange\nbased:\n●\nMetric\nchange\nquestions\ntest\nif\nyou\nknow\nwhat\nto\ndo\nwhen\na\nkey\nproduct\nmetric\n(e.g.,\ntraffic,\nrevenue,\nengagement,\netc.)\nis\ngoing\nup\nor\ndown\nfor\nno\napparent\nreason.\n●\nThis\nis\nalmost\nthe\nsame\nas\nroot\ncause\nanalysis\ni.e.:\nan\napproach\nfor\nidentifying\nthe\nunderlying\ncauses\nof\nan\nincident\nso\nthat\nthe\nmost\neffective\nsolutions\ncan\nbe\nidentified\nand\nsolved.\nWe\nwill\nbe\nfocusing\non\nmetric\ndefinition\nquestions\nhere\n-\nFramework\nfor\nworking\non\nsuch\nproblems\n-\nHere\nis\nhow\nyou\ncan\nanswer\na\nmetrics\ndefinition-based\nquestion:\n1.\nDescribe\nthe\nfeat

In [25]:
docs_mmr = db.max_marginal_relevance_search(query, k=4)
docs_mmr

[Document(metadata={'source': 'PA - Consolidated lecture notes.pdf', 'page': 20}, page_content='●\nThere\nare\nmany\ndifferent\nmetrics\nyou\ncould\nbe\ntracking\n(e.g.,\nimpressions,\nclicks,\nreturn\non\nad\nspend,\netc.)\nand\nyour\ninterviewer\nwill\nwant\nto\nhear\nyou\nselect\nthe\nmost\nimportant\nones\nusing\na\nrigorous\nprocess.\n2.\nMetric\nchange\nbased:\n●\nMetric\nchange\nquestions\ntest\nif\nyou\nknow\nwhat\nto\ndo\nwhen\na\nkey\nproduct\nmetric\n(e.g.,\ntraffic,\nrevenue,\nengagement,\netc.)\nis\ngoing\nup\nor\ndown\nfor\nno\napparent\nreason.\n●\nThis\nis\nalmost\nthe\nsame\nas\nroot\ncause\nanalysis\ni.e.:\nan\napproach\nfor\nidentifying\nthe\nunderlying\ncauses\nof\nan\nincident\nso\nthat\nthe\nmost\neffective\nsolutions\ncan\nbe\nidentified\nand\nsolved.\nWe\nwill\nbe\nfocusing\non\nmetric\ndefinition\nquestions\nhere\n-\nFramework\nfor\nworking\non\nsuch\nproblems\n-\nHere\nis\nhow\nyou\ncan\nanswer\na\nmetrics\ndefinition-based\nquestion:\n1.\nDescribe\nthe\nfeatu

In [5]:
loaders = [
    PyPDFLoader("PA - Consolidated lecture notes.pdf"),
    PyPDFLoader("SQL Revision Notes.pdf"),
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

splits = text_splitter.split_documents(docs)

split_texts = [str(doc.page_content) for doc in splits]

In [6]:
len(split_texts)

83

In [7]:
import chromadb
from chromadb.utils.embedding_functions import create_langchain_embedding

ef = create_langchain_embedding(embedding)

In [8]:
chroma_client = chromadb.Client()
client = chromadb.PersistentClient(path="database")

chroma_collection = chroma_client.create_collection("chroma_check_1", embedding_function=ef)
# This method creates a new collection within ChromaDB.
# Any document added to this collection will be processed using the ef function to generate its embedding.

In [9]:
ids = [str(i) for i in range(len(split_texts))] #creating ids for all splits

chroma_collection.add(ids=ids, documents=split_texts)
chroma_collection.count() # verify that all the documents were added successfully.


83

In [10]:
query = "what is group by clause in sql"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print((document))
    print('\n')

GROUP
BY
●
GROUP
BY
groups
together
rows
that
have
the
same
values
in
specified
columns.
It
computes
summaries
(aggregated
measures)
for
each
group
using
the
aggregate
functions.
●
Syntax:
SELECT
[column_1],
[column_2],
COUNT()
AS
[column_name]
SUM()
AS
[column_name]
FROM
[table_name]
WHERE
[condition]
GROUP
BY
[column_1],
[column_2];
●
Example
1:
Get
the
number
of
purchases
each
customer
has
made
on
each
market
date.
Use
the
customer_purchases
table
from
the
Farmer’s
Market(Mandi)
database
schema.
Solution
query:
SELECT
market_date,
customer_id,
count(*)
AS
num_purchases
FROM
customer_purchases
GROUP
BY
market_date,
customer_id
ORDER
BY
market_date
Output:
GROUP
BY
vs.
DISTINCT
●
DISTINCT
is
used
to
filter
unique
records
out
of
all
records
in
the
table.
It
removes
the 
duplicate
rows.


●
The
GROUP
BY
clause
is
used
to
group
a
selected
set
of
rows
into
summary
rows
by
one 
or
more
columns
or
an
expression. 
●
The
group
by
gives
the
same
result
as
distinct
when
no
aggregate
function
is

In [11]:
## Using open source llama 3 model from TogetherAI 

from langchain_together import ChatTogether

llm = ChatTogether(
    api_key = os.getenv("TOGETHER_API_KEY"),
    temperature=0.0, 
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"
)

In [12]:
def rag(query, retrieved_documents, llm=llm):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a helpful SQL assistant. Your users are asking questions about information contained in a report."
            "You will be shown the user's question, and the relevant information from the report. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]

    response = llm.invoke(messages)

    content = response.content
    return content

In [13]:
output = rag(query=query, retrieved_documents=retrieved_documents)

print((output))

The GROUP BY clause in SQL is used to group together rows that have the same values in specified columns. It computes summaries (aggregated measures) for each group using aggregate functions. The GROUP BY clause is used to group a selected set of rows into summary rows by one or more columns or an expression.


In [14]:
query = "how to pursure product manager role"
results = chroma_collection.query(
    query_texts=query,
    n_results=8,
    include=["documents", "embeddings"])

retrieved_documents = results["documents"][0]

for document in results["documents"][0]:
    print(document)
    print("")

Product
Sense
-
How
to
tackle
product
strategy
and
business
acumen
rounds
in
interviews?
Lecture
Objective:
How
to
address
business
acumen
questions
round
:
●
Analyzing
a
metric
change.
What’s
a
metric?
●
Deﬁning
metrics
to
measure
performance
/
success
of
a
new
feature
/
product.
Importance
of
product
strategy
&
business
acumen
rounds
-
●
Apart
from
building
models,
developing
dashboards
and
reporting
frameworks
-
One
of
the
main
responsibilities
of
a
data
scientist
is
to
extract
insights
from
data
and
work
with
product
managers
and
engineering
teams
to
deliver
actionable
plans
to
i
mprove
the
product.
●
Product
sense
is
about
understanding
all
possibilities
,
not
ﬁnding
one
correct
answer.
Example
Questions:
Product
Acumen
/
Business
Acumen
●
Why
did
Youtube’s
trafﬁc
drop
by
5%?
●
How
would
you
measure
the
success
of
the
“Save
Post”
feature
on
Facebook?
●
What
metrics
would
you
deﬁne
to
measure
the
health
of
the
product
search
in
Amazon?
●
We
have
a
dashboard
tracking
our
metrics
and

In [20]:
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction 
from sentence_transformers import CrossEncoder # ! pip install sentence-transformers

cross_encoder = CrossEncoder("BAAI/bge-reranker-v2-m3")

OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\Varun.Tyagi\OneDrive - Brillio\Documents\workspace\GenAI\genai_env\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.